In [1]:
import torch

torch.cuda.is_available()

True

# 数据
在mmpretrain下创建data目录，将水果图片数据（30类）放入其中。

In [2]:
%cd mmpretrain
!mkdir data

f:\a_deskFile\OpenMMLab实战营第二期\MMPretrain\mmpretrain


# 配置文件
## 查询配置文件

In [3]:
from mmpretrain import get_model, list_models, inference_model

# 列车图片分类任务中，resnet18模型有哪些
print(list_models(task='Image Classification', pattern='resnet50'))


['resnet50_8xb16_cifar10', 'resnet50_8xb16_cifar100', 'resnet50_8xb256-rsb-a1-600e_in1k', 'resnet50_8xb256-rsb-a2-300e_in1k', 'resnet50_8xb256-rsb-a3-100e_in1k', 'resnet50_8xb32-fp16_in1k', 'resnet50_8xb32_in1k', 'resnet50_8xb8_cub', 'resnet50_barlowtwins-pre_8xb32-linear-coslr-100e_in1k', 'resnet50_byol-pre_8xb512-linear-coslr-90e_in1k', 'resnet50_densecl-pre_8xb32-linear-steplr-100e_in1k', 'resnet50_mocov2-pre_8xb32-linear-steplr-100e_in1k', 'resnet50_mocov3-100e-pre_8xb128-linear-coslr-90e_in1k', 'resnet50_mocov3-300e-pre_8xb128-linear-coslr-90e_in1k', 'resnet50_mocov3-800e-pre_8xb128-linear-coslr-90e_in1k', 'resnet50_simclr-200e-pre_8xb512-linear-coslr-90e_in1k', 'resnet50_simclr-800e-pre_8xb512-linear-coslr-90e_in1k', 'resnet50_simsiam-100e-pre_8xb512-linear-coslr-90e_in1k', 'resnet50_simsiam-200e-pre_8xb512-linear-coslr-90e_in1k', 'resnet50_swav-pre_8xb32-linear-coslr-100e_in1k']


In [4]:
# 选择模型 ： resnet50_8xb16_cifar100
# 从官网拿到模型链接：https://download.openmmlab.com/mmclassification/v0/resnet/resnet50_b16x8_cifar100_20210528-67b58a1b.pth
# 模型实例化
model = get_model('resnet50_8xb16_cifar100')
print(type(model))
print(type(model.backbone))

<class 'mmpretrain.models.classifiers.image.ImageClassifier'>
<class 'mmpretrain.models.backbones.resnet_cifar.ResNet_CIFAR'>


## 配置文件  resnet50_8xb16_cifar100.py
对文件进行适配性修改：\
```python

# 这段改成我的配置文件路径
'''
_base_ = [
    '../_base_/models/resnet50_cifar.py',
    '../_base_/datasets/cifar100_bs16.py',
    '../_base_/schedules/cifar10_bs128.py',
    '../_base_/default_runtime.py',
]
'''

_base_ = [
    '../../../config_for_fruits.py',
]


# model settings-------------------
# 由于我们的分类只有30类，所以num_classes=30
model = dict(head=dict(num_classes=30))

# schedule settings-----------------
optim_wrapper = dict(optimizer=dict(weight_decay=0.0005))


# data settings---------------------

# 数据集类型为CustomDataset
dataset_type = 'CustomDataset'

# 同理，我们只有30类
data_preprocessor = dict(num_classes=30)

# 训练集路径改成自己的
train_dataloader = dict(dataset=dict(data_root='data/fruit30_train'))

# 验证集也是的
val_dataloader = dict(dataset=dict(data_root='data/fruit30_train'))

# 再配置文件中添加一个test_dataloader，因为原来的配置文件中没有

# 规划配置
param_scheduler = dict(
    type='MultiStepLR',
    by_epoch=True,
    milestones=[60, 120, 160],
    gamma=0.2,
)


# 由于是微调，只需少量训练即可，我设置为7
train_cfg = dict(by_epoch=True, max_epochs=7, val_interval=1)

# lr调小一点
optim_wrapper = dict(
    optimizer=dict(type='SGD', lr=0.01, momentum=0.9, weight_decay=0.0001))


# 加载预训练权重
init_cfg=dict(type='Pretrained', checkpoint='https://download.openmmlab.com/mmclassification/v0/resnet/resnet50_b16x8_cifar100_20210528-67b58a1b.pth')

```

## 配置文件tips：
1. split字段删除；
2. 

# 加载预训练权重
在model字典里面修改添加字段：
```python
init_cfg=dict(type='Pretrained', checkpoint='https://download.openmmlab.com/mmclassification/v0/resnet/resnet50_b16x8_cifar100_20210528-67b58a1b.pth')
```

## 训练中的几个问题
1. 在训练中出现了dataloader有问题的情况，由于不熟悉customdataset和imagenet数据集，所以按照视频中老师使用的imagenet_bs32.py基础上进行修改。  \
经过查询，数据集的类型主要分为上述两种，对于customdataset类型，有子文件夹方式（就是我们那种）和标注文件方式（文本标注文件来存储图像文件路径和对应的类别索引）。 \
更多信息见：[准备数据集](https://mmpretrain.readthedocs.io/zh_CN/latest/user_guides/dataset_prepare.html#id2) \
但是，如果选择的模型类型不好，按照网站说明进行修改也是没用的。

2. 出现cuda out of memory。先可以尝试修改batch_size等参数。 \
如果还是容易显存溢出，那么还是模型选择的问题，即模型本身太复杂。

综上：我更换了下面两个配置文件：
cifar100_bs16.py ——————》 imagenet_bs32.py  \
resnet50_cifar.py ——————》 resnet50.py

# 启动训练

In [5]:
!mim train mmpretrain config_for_fruits01.py --work-dir=./exp

^C


# 开启测试

In [ ]:
!mim test mmpretrain config_for_fruits01.py --checkpoint ./exp/epoch_7.pth --out result.pkl

# 分析结果

In [ ]:
!mim run mmpretrain analyze_results config_for_fruits01.py result.pkl --out-dir analyze
!mim run mmpretrain confusion_matrix config_for_fruits01.py result.pkl --show --show-path ./matrix.png --include-values

# 最后，在一个真实图片上进行推理

In [4]:
from mmpretrain import ImageClassificationInferencer

inferencer = ImageClassificationInferencer('./mmpretrain/config_for_fruits01.py', pretrained='./mmpretrain/exp/epoch_7.pth')
inferencer("./葡萄.jpg", show=True)

Loads checkpoint by local backend from path: ./mmpretrain/exp/epoch_7.pth


f:\Anaconda\envs\mmpretrain\lib\site-packages\mmengine\visualization\utils.py:240: UserWarning: Glyph 33889 (\N{CJK UNIFIED IDEOGRAPH-8461}) missing from current font.
  s, (width, height) = canvas.print_to_buffer()
f:\Anaconda\envs\mmpretrain\lib\site-packages\mmengine\visualization\utils.py:240: UserWarning: Glyph 33796 (\N{CJK UNIFIED IDEOGRAPH-8404}) missing from current font.
  s, (width, height) = canvas.print_to_buffer()
f:\Anaconda\envs\mmpretrain\lib\site-packages\mmengine\visualization\utils.py:240: UserWarning: Glyph 30333 (\N{CJK UNIFIED IDEOGRAPH-767D}) missing from current font.
  s, (width, height) = canvas.print_to_buffer()


[{'pred_scores': array([4.3139076e-03, 4.9760711e-04, 3.8070935e-03, 1.1619236e-03,
         1.0601275e-02, 1.1192159e-02, 1.7339033e-03, 1.6992692e-02,
         2.3214186e-03, 3.0637996e-03, 3.7959507e-03, 1.1862597e-01,
         1.4049634e-03, 1.5994417e-04, 9.3369518e-04, 2.7724623e-04,
         1.1717105e-02, 1.6713761e-01, 7.8622956e-04, 3.8845200e-02,
         9.7611157e-04, 1.3152322e-03, 1.8268734e-03, 5.6421590e-01,
         3.0788119e-04, 2.8142405e-03, 1.0715317e-03, 4.2732293e-04,
         2.2278760e-02, 5.3964667e-03], dtype=float32),
  'pred_label': 23,
  'pred_score': 0.564215898513794,
  'pred_class': '葡萄-白'}]

In [5]:
inferencer("./红葡萄.jpg", show=True)

f:\Anaconda\envs\mmpretrain\lib\site-packages\mmengine\visualization\utils.py:240: UserWarning: Glyph 32418 (\N{CJK UNIFIED IDEOGRAPH-7EA2}) missing from current font.
  s, (width, height) = canvas.print_to_buffer()


[{'pred_scores': array([1.0910064e-05, 6.9411611e-03, 1.6560815e-02, 3.0406899e-04,
         2.1750839e-02, 2.1583910e-05, 7.8190524e-06, 9.6244941e-05,
         2.4509376e-05, 6.6590706e-06, 1.8346560e-01, 6.1708415e-05,
         1.2899390e-02, 9.4035429e-05, 4.1310024e-04, 6.2147279e-07,
         1.3216780e-03, 1.4125090e-04, 1.4464279e-01, 7.8748752e-05,
         9.5033012e-03, 3.8337141e-02, 6.0287075e-06, 5.0685660e-04,
         5.0527364e-01, 2.1797428e-02, 2.9431761e-03, 3.2691777e-02,
         5.6309098e-05, 4.0794708e-05], dtype=float32),
  'pred_label': 24,
  'pred_score': 0.5052736401557922,
  'pred_class': '葡萄-红'}]

In [6]:
inferencer("./火龙果.jpg", show=True)

f:\Anaconda\envs\mmpretrain\lib\site-packages\mmengine\visualization\utils.py:240: UserWarning: Glyph 28779 (\N{CJK UNIFIED IDEOGRAPH-706B}) missing from current font.
  s, (width, height) = canvas.print_to_buffer()
f:\Anaconda\envs\mmpretrain\lib\site-packages\mmengine\visualization\utils.py:240: UserWarning: Glyph 40857 (\N{CJK UNIFIED IDEOGRAPH-9F99}) missing from current font.
  s, (width, height) = canvas.print_to_buffer()
f:\Anaconda\envs\mmpretrain\lib\site-packages\mmengine\visualization\utils.py:240: UserWarning: Glyph 26524 (\N{CJK UNIFIED IDEOGRAPH-679C}) missing from current font.
  s, (width, height) = canvas.print_to_buffer()


[{'pred_scores': array([4.2940243e-04, 3.8071728e-04, 1.2785956e-03, 6.8414665e-05,
         3.0214179e-03, 1.6001533e-03, 9.8025288e-05, 1.3903201e-03,
         2.8057478e-04, 3.0066262e-04, 8.6054957e-01, 2.7169302e-04,
         1.9328629e-03, 3.6561862e-04, 2.2745530e-04, 2.5972529e-05,
         1.5738173e-03, 7.8223260e-05, 1.0373752e-01, 1.1104680e-04,
         4.1459640e-03, 2.6031872e-03, 7.6874444e-04, 5.4993550e-03,
         1.9316936e-03, 2.0142104e-03, 2.5246275e-04, 4.2833514e-03,
         6.9571752e-04, 8.3243693e-05], dtype=float32),
  'pred_label': 10,
  'pred_score': 0.8605495691299438,
  'pred_class': '火龙果'}]

In [ ]:
'''
from configs._base_.models import resnet50_cifar
from configs._base_.datasets import cifar100_bs16
from configs._base_.schedules import cifar10_bs128
from configs._base_ import default_runtime
'''